# Dataset Processing 
Using scVelo built in datasets for pancreas, hindbrain, and denategyrus. Over 10k cells across 3 tissues.

In [1]:
!pip install scvelo

## General Preprocessing Pipeline

In [ ]:
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np

"""
Purpose: Visualize if the datasets are good enough for preprocessing
Params: The file path to the dataset
Returns: None
"""

def process_dataset(filepath):
    # Read in file 
    if filepath.endswith('.loom'):
        adata = sc.read_loom(filepath)
    elif filepath.endswith('.h5ad'):
        adata = sc.read_h5ad(filepath)
    else:
        try:
            adata = scv.read(filepath, cache=True)
        except AttributeError:
            adata = sc.read(filepath)

    # Validate scVelo usage
    print("Layers:", adata.layers.keys()) # print all layers included
    print("Has spliced:", "spliced" in adata.layers, "\nHas unspliced:", "unspliced" in adata.layers) # check necessary layers for RNA velocity

    #Visualize Counts
    print("Spliced matrix preview:")
    print(adata.layers["spliced"][:5, :5])

    print("Unspliced matrix preview:")
    print(adata.layers["unspliced"][:5, :5])

    # Validate dimensions (genes x cells)
    print("Shape:", adata.shape)

    # Check for cell metadata
    print("OBS columns:", adata.obs.columns.tolist())
    display(adata.obs.head())

    # Check for gene metadata
    print("VAR columns:", adata.var.columns.tolist())
    display(adata.var.head())

    # Total counts
    total_counts = np.sum(adata.layers["spliced"], axis=1)
    print("UMI stats (min, max, mean):", total_counts.min(), total_counts.max(), total_counts.mean())


## Embryogenesis data
Link: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE186068


In [ ]:
#process_dataset("embryo/embryogenesis.loom")

: 

## Further Exploration with Embryonic Dataset - Blood (and Neuron later)

In [2]:
import scanpy as sc
import pandas as pd
from scipy.io import mmread
import numpy as np

"""
Purpose: Verify the matrices and counts for the trajectory chosen
Params: dataset name (blood or neurons)
Return: introns, extrons, var, obs
"""
def verify_trajectory(dataset): 
    print("Load the MOCA dataset subset used by TOME for subset trajectory")

    #1. Load matrices ( cells x genes)
    print("\nLoading count matrices:")
    exon = mmread(f"{dataset}/exp_exon.mtx").tocsr()
    intron = mmread(f"{dataset}/exp_intron.mtx").tocsr()
    print(f"Spliced: {exon.shape}")
    print(f"Unspliced: {intron.shape}")

    #2. Load cell metadata
    print("\nLoading cell metadata:")
    obs = pd.read_csv(f"{dataset}/obs.csv", index_col=0)
    print(f"Cells: {len(obs):,}")
    print(f"Columns: {list(obs.columns)}")

    #3. Load gene metadata
    print("\nLoading gene metadata:")
    var = pd.read_csv("E9.5_to_E13.5_var.csv", index_col=0) #same gene file across 
    print(f"Genes: {len(var):,}")

    #4. Verify alignment
    print("\nVerify the dimensions dimensions:")
    assert exon.shape[0] == len(obs), f"Cell mismatch: {exon.shape[0]} vs {len(obs)}"
    assert exon.shape[1] == len(var), f"Gene mismatch: {exon.shape[1]} vs {len(var)}"
    print("All dimensions aligned!")

    return intron, exon, var, obs



In [3]:
"""
Purpose: Convert trajectory information into adata
Params: intron, extron, var, obs files
Return: adata object
"""

def convert_trajectory(intron, exon, var, obs):
    # Get the adata file for easy access
    adata = sc.AnnData(X=(exon + intron).tocsr())

    # Add cell metadata
    adata.obs = obs.copy()
    adata.obs_names = obs.index.astype(str)

    # Add gene metadata  
    adata.var = var.copy()
    adata.var_names = var['gene_short_name'].values
    adata.var_names_make_unique()

    # Add spliced/unspliced layers for velocity
    adata.layers['spliced'] = exon
    adata.layers['unspliced'] = intron


    # Get numeric timepoint from 'day' column
    adata.obs['timepoint_str'] = adata.obs['day'].astype(str)
    adata.obs['timepoint'] = (
        adata.obs['day']
        .str.replace('E', '', regex=False)
        .str.replace('b', '', regex=False)  # Handle E8.5b -> 8.5
        .astype(float)
    )

    return adata


"""
Purpose: Summarize and save data
Params: adata object, dataset name
Return: None
"""
def save_adata(adata, dataset):
    # Print final summary
    print("\nFINAL ANNDATA SUMMARY")
    print("-" * 50)
    print(f"Cells: {adata.n_obs:,}")
    print(f"Genes: {adata.n_vars:,}")
    print(f"Layers: {list(adata.layers.keys())}")
    print(f"Obs cols: {list(adata.obs.columns)}")

    print(f"\nTimepoints:")
    print(adata.obs['timepoint'].value_counts().sort_index())

    print(f"\nCell types:")
    print(adata.obs['celltype'].value_counts())

    # Save file
    adata.write(f"{dataset}_velocity_ready.h5ad")
    print(f"Saved: {dataset}_velocity_ready.h5ad")


In [4]:
"""
Purpose: create and save file adata file!
Params: dataset name
Return: None
"""
def create_adata(dataset):
    intron, extron, var, obs = verify_trajectory(dataset)
    adata = convert_trajectory(intron, extron, var, obs)
    save_adata(adata, dataset)

create_adata('blood')

Load the MOCA dataset subset used by TOME for subset trajectory

Loading count matrices:
Spliced: (53268, 24552)
Unspliced: (53268, 24552)

Loading cell metadata:
Cells: 53,268
Columns: ['Anno', 'day', 'celltype', 'sample', 'batch', 'group']

Loading gene metadata:
Genes: 24,552

Verify the dimensions dimensions:
All dimensions aligned!

FINAL ANNDATA SUMMARY
--------------------------------------------------
Cells: 53,268
Genes: 24,552
Layers: ['spliced', 'unspliced']
Obs cols: ['Anno', 'day', 'celltype', 'sample', 'batch', 'group', 'timepoint_str', 'timepoint']

Timepoints:
timepoint
8.5      2877
9.5      3390
10.5     9308
11.5    14930
12.5     9090
13.5    13673
Name: count, dtype: int64

Cell types:
celltype
Definitive erythroid cells    22038
Primitive erythroid cells     21309
White blood cells              8213
Megakaryocytes                 1509
Blood progenitors               199
Name: count, dtype: int64
Saved: blood_velocity_ready.h5ad


## Graph Creation
Take adata and convert it to a pytorch graph. Initially do cosine similarity between gene expression for the edge weights. Will re-weight with scVelo RNA velocity down the line. Let timepoints be node features as well as gene expression.

In [7]:
%pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster torch-spline-conv -y
%pip install torch


Found existing installation: torch-scatter 2.1.2
Uninstalling torch-scatter-2.1.2:
  Successfully uninstalled torch-scatter-2.1.2
Found existing installation: torch-sparse 0.6.18
Uninstalling torch-sparse-0.6.18:
  Successfully uninstalled torch-sparse-0.6.18
Found existing installation: torch_geometric 2.4.0
Uninstalling torch_geometric-2.4.0:
  Successfully uninstalled torch_geometric-2.4.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%%python -c "import torch; print(torch.__version__)"
%pip install --upgrade pip

2.5.1


In [10]:
%pip install torch-geometric
%pip install scanpy

  Using cached torch_geometric-2.7.0-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.7.0-py3-none-any.whl (1.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torch-geometric] [torch-geometric]
Note: you may need to restart the kernel to use updated packages.
  Using cached scanpy-1.11.5-py3-none-any.whl.metadata (9.3 kB)
  Using cached anndata-0.12.6-py3-none-any.whl.metadata (10.0 kB)
  Using cached h5py-3.15.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached legacy_api_wrap-1.5-py3-none-any.whl.metadata (2.2 kB)
  Using cached matplotlib-3.10.7-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached numba-0.62.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.8 kB)
  Using cached patsy-1.0.2-py2.py3-none-any.whl.metadata (3.6 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cach

In [ ]:
# !pip install "numpy<2"
%pip install scanpy


Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
import scipy.sparse as sp
import scanpy as sc

# Load the adata 
adata = sc.read_h5ad("blood_velocity_ready.h5ad")
print(f"Loaded: {adata.n_obs:,} cells and {adata.n_vars:,} genes")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded: 53,268 cells and 24,552 genes


In [ ]:
"""
Purpose: create the graph as a pytorch graph and save 
Params: dataset name, adata object, preprocessing params
Return: None
"""
def create_graph(dataset, adata, n_neighbors=15, n_comps=50, n_pcs=50, n_top_genes=2000):
    """Data preprocessing for PCA"""
    # Standard filtering and normalization
    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)

    print(f"Highly variable genes: {adata.var['highly_variable'].sum()}")

    sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata, n_comps=n_comps, use_highly_variable=True)
    expr_pca = adata.obsm['X_pca']  # [n_cells, 50]
    print(f"PCA shape: {expr_pca.shape}")

    """Adding Node Features"""
    # PCA components --> gene expresion only
    x = torch.tensor(expr_pca, dtype=torch.float32)
    print(f"      Node features: {x.shape[1]} (PCA components)")

    # Timepoint as separate attribute
    timepoints = torch.tensor(adata.obs['timepoint'].values, dtype=torch.float32)

    # Also store normalized version for models that want it
    timepoints_norm = (timepoints - timepoints.mean()) / timepoints.std()

    print(f"Timepoints: {timepoints.unique().sort()[0].tolist()}")

    """Adding Edgess"""

    # Compute neighbors w/ PCA space
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)

    connectivities = adata.obsp['connectivities']
    rows, cols = connectivities.nonzero()

    # Cosine similarity on PCA embeddings for edge weights
    print("Running cosine similarities")
    edge_weights = []
    for i, j in zip(rows, cols):
        sim = cosine_similarity(expr_pca[i:i+1], expr_pca[j:j+1])[0, 0]
        edge_weights.append(sim)

    edge_weights = np.array(edge_weights)

    edge_index = torch.tensor(np.vstack([rows, cols]), dtype=torch.long)
    edge_attr = torch.tensor(edge_weights, dtype=torch.float32).unsqueeze(1)

    print(f"Edges: {edge_index.shape[1]:,}")
    print(f"Edge weights: [{edge_weights.min():.3f}, {edge_weights.max():.3f}]")

    """Adding Metadata (Cell Labels)"""
    celltype_cat = adata.obs['celltype'].astype('category')
    celltype_codes = torch.tensor(celltype_cat.cat.codes.values, dtype=torch.long)
    celltype_names = list(celltype_cat.cat.categories)

    print(f"Cell types: {len(celltype_names)}")

    """Attempting THIS GRAPHHHH"""
    try:
        graph = Data(
            x=x,                              # [n_cells, 50] PCA embeddings
            edge_index=edge_index,            # [2, n_edges]
            edge_attr=edge_attr,              # [n_edges, 1] cosine similarity
            
            # Separate node attributes
            timepoint=timepoints,             # [n_cells] raw timepoint (e.g., 8.5, 10.5)
            timepoint_norm=timepoints_norm,   # [n_cells] normalized timepoint
            celltype=celltype_codes,          # [n_cells] integer cell type labels
        )
        
        # Store metadata (not tensors)
        graph.celltype_names = celltype_names
        graph.n_cells = adata.n_obs
        graph.n_pcs = 50
        
        
        print("Graph created:", graph)

    except ImportError:
        graph = {
            'x': x,
            'edge_index': edge_index,
            'edge_attr': edge_attr,
            'timepoint': timepoints,
            'timepoint_norm': timepoints_norm,
            'celltype': celltype_codes,
            'celltype_names': celltype_names,
        }
        print(f"Graph dict created instead ig....")

    # Save graph as pytorch graph and rewrite adata
    torch.save(graph, f"{dataset}_graph.pt")
    print(f"\n Saved graph")

    adata.write(f"{dataset}_velocity_ready.h5ad")
    print("Updated .h5ad file")


In [18]:
create_graph('blood',adata, n_neighbors=15, n_comps=50, n_pcs=50, n_top_genes=2000)

Highly variable genes: 2000


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/functools.py:909: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scanpy/preprocessing/_pca/__init__.py:226: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  mask_var_param, mask_var = _handle_mask_var(


PCA shape: (53268, 50)
      Node features: 50 (PCA components)
Timepoints: [8.5, 9.5, 10.5, 11.5, 12.5, 13.5]
Running cosine similarities
Edges: 1,343,922
Edge weights: [0.127, 0.989]
Cell types: 5
Graph created: Data(x=[53268, 50], edge_index=[2, 1343922], edge_attr=[1343922, 1], timepoint=[53268], timepoint_norm=[53268], celltype=[53268], celltype_names=[5], n_cells=53268, n_pcs=50)

 Saved graph
Updated .h5ad file


## Reweight the edges
Use scVelo and the equation cosine_w + alpha * cosine_similarity(distance between cells, RNA velocity). This should downweight cells not in the direction of the RNA velocity, while upweighting those that are...

In [20]:
%pip install scvelo

  Using cached scvelo-0.3.3-py3-none-any.whl.metadata (8.0 kB)
  Using cached loompy-3.0.8-py3-none-any.whl
  Using cached numpy_groupies-0.11.3-py3-none-any.whl.metadata (18 kB)
Using cached scvelo-0.3.3-py3-none-any.whl (196 kB)
Using cached numpy_groupies-0.11.3-py3-none-any.whl (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scvelo]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import scvelo as scv
import scanpy as sc
import numpy as np
import torch

"""
Purpose: Compute RNA velocity using scVelo
Params: adata object, preprocessing parameters
Return: adata with velocity computed
"""
def compute_velocity(adata, min_shared_counts=20, n_top_genes=2000, n_pcs=50, n_neighbors=15):
    print("COMPUTING RNA VELOCITY")
    
    # scVelo preprocessing
    print("\nPreprocessing for velocity...")
    scv.pp.filter_and_normalize(adata, min_shared_counts=min_shared_counts, n_top_genes=n_top_genes)
    scv.pp.moments(adata, n_pcs=n_pcs, n_neighbors=n_neighbors)
    
    # Compute velocity
    print("\nComputing velocity...")
    scv.tl.velocity(adata, mode='stochastic')
    
    # Build velocity graph
    print("\nBuilding velocity graph...")
    scv.tl.velocity_graph(adata)
    
    # Project velocity to PCA space manually
    print("\nProjecting velocity to PCA space...")
    
    # Get the velocity in gene space
    velocity_genes = adata.layers['velocity']
    
    # Get the PCA loadings (genes x PCs)
    pca_loadings = adata.varm['PCs']  # [n_genes, n_pcs]
    
    # Project: velocity_pca = velocity_genes @ pca_loadings
    # Handle sparse matrix if needed
    if hasattr(velocity_genes, 'toarray'):
        velocity_genes = velocity_genes.toarray()
    
    # Replace NaNs with 0 (some genes have no velocity)
    velocity_genes = np.nan_to_num(velocity_genes, nan=0.0)
    
    velocity_pca = velocity_genes @ pca_loadings
    adata.obsm['velocity_pca'] = velocity_pca
    
    print(f"\n✓ Velocity computed!")
    print(f"  Velocity (genes): {adata.layers['velocity'].shape}")
    print(f"  Velocity (PCA):   {velocity_pca.shape}")
    
    return adata


"""
Purpose: Reweight graph edges using velocity alignment
Params: graph object, adata with velocity, alpha hyperparameter
Return: graph with updated edge weights
"""
def reweight_edges(graph, adata, alpha=0.5):
    print("REWEIGHTING EDGES WITH VELOCITY")
    print("-" * 50)
    
    # Get velocity in PCA space
    velocity_pca = adata.obsm['velocity_pca']
    
    # Get graph data as numpy
    x = graph.x.numpy()
    edge_index = graph.edge_index.numpy()
    w_cosine = graph.edge_attr.numpy().flatten()
    
    print(f"\nAlpha: {alpha}")
    print(f"Edges: {edge_index.shape[1]:,}")
    print("Computing velocity alignment...")
    
    n_edges = edge_index.shape[1]
    alignments = np.zeros(n_edges)
    
    for idx in range(n_edges):
        i, j = edge_index[0, idx], edge_index[1, idx]
        
        # Displacement: direction from cell i to cell j
        d_ij = x[j] - x[i]
        
        # Velocity: where cell i is heading
        v_i = velocity_pca[i]
        
        # Cosine similarity between velocity and displacement
        norm_d = np.linalg.norm(d_ij)
        norm_v = np.linalg.norm(v_i)
        
        if norm_d > 0 and norm_v > 0:
            alignments[idx] = np.dot(v_i, d_ij) / (norm_v * norm_d)
        else:
            alignments[idx] = 0
    
    # Compute new weights
    w_new = (1 - alpha) * w_cosine + alpha * alignments
    
    # Print stats
    print(f"\nAlignment stats:")
    print(f"  Range: [{alignments.min():.3f}, {alignments.max():.3f}]")
    print(f"  Mean:  {alignments.mean():.3f}")
    
    print(f"\nEdge weight stats:")
    print(f"  Before: [{w_cosine.min():.3f}, {w_cosine.max():.3f}]")
    print(f"  After:  [{w_new.min():.3f}, {w_new.max():.3f}]")
    
    # Update graph
    graph.edge_attr = torch.tensor(w_new, dtype=torch.float32).unsqueeze(1)
    graph.velocity_alignment = torch.tensor(alignments, dtype=torch.float32)
    
    print(f"\nEdges reweighted!")
    
    return graph


"""
Purpose: Save velocity adata and reweighted graph
Params: adata, graph, dataset name
Return: None
"""
def save_velocity_data(adata, graph, dataset):
    print("\nSAVING NOW")
    
    adata.write(f"{dataset}_velocity_computed.h5ad")
    print(f"Saved: {dataset}_velocity_computed.h5ad")
    
    torch.save(graph, f"{dataset}_graph_velocity.pt")
    print(f"Saved: {dataset}_graph_velocity.pt")


"""
Purpose: Full pipeline to compute velocity and reweight graph
Params: adata, dataset name, alpha, preprocessing params
Return: adata, graph (both updated)
"""
def velocity_pipeline(adata, dataset, alpha=0.5, min_shared_counts=20, n_top_genes=2000, n_pcs=50, n_neighbors=15):
    # Load existing graph
    graph = torch.load(f"{dataset}_graph.pt", weights_only=False)
    print(f"Loaded graph: {graph.x.shape[0]:,} nodes, {graph.edge_index.shape[1]:,} edges")
    
    # Compute velocity
    adata = compute_velocity(
        adata,
        min_shared_counts=min_shared_counts,
        n_top_genes=n_top_genes,
        n_pcs=n_pcs,
        n_neighbors=n_neighbors
    )
    
    # Reweight edges
    graph = reweight_edges(graph, adata, alpha=alpha)
    
    # Save
    save_velocity_data(adata, graph, dataset)
    
    return adata, graph


In [5]:
# Load the adata 
adata = sc.read_h5ad("blood_velocity_ready.h5ad")
new_adata, new_graph = velocity_pipeline(adata, 'blood', alpha=0.5, min_shared_counts=20, n_top_genes=2000, n_pcs=50, n_neighbors=15)
   

Loaded graph: 53,268 nodes, 1,343,922 edges
COMPUTING RNA VELOCITY

Preprocessing for velocity...
Filtered out 8520 genes that are detected 20 counts (shared).
Normalized count data: spliced, unspliced.
Extracted 6801 highly variable genes.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


Logarithmized X.
computing moments based on connectivities
    finished (0:00:06) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)

Computing velocity...
computing velocities


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scvelo/tools/optimization.py:184: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gamma[i] = np.linalg.pinv(A.T.dot(A)).dot(A.T.dot(y[:, i]))


    finished (0:00:30) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)

Building velocity graph...
computing velocity graph (using 1/8 cores)
or disable the progress bar using `show_progress_bar=False`.
    finished (0:01:30) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)

Projecting velocity to PCA space...

✓ Velocity computed!
  Velocity (genes): (53268, 6801)
  Velocity (PCA):   (53268, 50)
REWEIGHTING EDGES WITH VELOCITY
--------------------------------------------------

Alpha: 0.5
Edges: 1,343,922
Computing velocity alignment...

Alignment stats:
  Range: [-0.846, 0.806]
  Mean:  -0.068

Edge weight stats:
  Before: [0.127, 0.989]
  After:  [-0.203, 0.861]

Edges reweighted!

SAVING NOW
Saved: blood_velocity_computed.h5ad
Saved: blood_graph_velocity.pt
